<a href="https://colab.research.google.com/github/Edward-TL/amazon_scraper/blob/master/amazon_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Amazon Web Scrapper



Se realiza la importación de requests y BeautifulSoup para poder extraer y ordenar la información del HTML y XPath. Se deja el diccionario de Direcciones para ajustar a un futuro las condiciones de la página.

In [55]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import os
import lxml.html as html

In [56]:
URL_dict = {'AMZ_Mex' : 'https://www.amazon.com.mx/',
            'top_kindle' : 'https://www.amazon.com.mx/gp/bestsellers/digital-text/ref=zg_bs_digital-text_home_all?pf_rd_p=01d2df49-74bd-42d3-a898-95a4f3beb031&pf_rd_s=center-4&pf_rd_t=2101&pf_rd_i=home&pf_rd_m=AVDBXBAVVSXLQ&pf_rd_r=6APAMAKKFSPVHG8MWSXT&pf_rd_r=6APAMAKKFSPVHG8MWSXT&pf_rd_p=01d2df49-74bd-42d3-a898-95a4f3beb031'}

Se selecciona la dirección que se va a obtener y se confirma el Estado de la petición al servidor.


In [57]:
URL = URL_dict['top_kindle']
print(f'URL given: {URL}')
response = requests.get(URL)
print(f'Request status: {response.status_code}')

URL given: https://www.amazon.com.mx/gp/bestsellers/digital-text/ref=zg_bs_digital-text_home_all?pf_rd_p=01d2df49-74bd-42d3-a898-95a4f3beb031&pf_rd_s=center-4&pf_rd_t=2101&pf_rd_i=home&pf_rd_m=AVDBXBAVVSXLQ&pf_rd_r=6APAMAKKFSPVHG8MWSXT&pf_rd_r=6APAMAKKFSPVHG8MWSXT&pf_rd_p=01d2df49-74bd-42d3-a898-95a4f3beb031
Request status: 200


In [58]:
from urllib.request import urlopen
from lxml import etree

Para garantizar la funcionalidad del scrapper, se condiciona para que corra unicamente en caso de tener una conexión existosa (Stadus Code: 200). De lo contrario, que nos informe el error.

#Obtención de la sopa

In [59]:
s = BeautifulSoup(response.text, 'lxml')

In [89]:
type(s)

bs4.BeautifulSoup

In [ ]:
print(s.prettify())

##Propiedades

###Ranking

In [ ]:
ranks_soup = s.find_all('span', attrs={'class':'zg-badge-text'})
ranks = [rank.get_text() for rank in ranks_soup]
ranks

###División del producto con imágen

In [ ]:
products_and_image = s.find_all('div', attrs={'class' : 'a-section a-spacing-small'})
products_and_image

###Nombre de los productos

In [ ]:
product_names = [product_name.img.get('alt') for product_name in products_and_image]
print(f'Nombres guardados: {len(product_names)}\n')
product_names

###URLs de las imágenes

In [ ]:
image_urls = [image_url.img.get('src') for image_url in products_and_image]
print(f'Nombres guardados: {len(image_urls)}\n')
image_urls

In [69]:
# product_names = []
# product_images_urls = []
# for product in products_and_image:
#     product_names = products_and_image.re(r'(?<=img\salt=")(.+)(?="\sheight)')
#     product_images_urls = products_and_image.re(r'(?<=src=")(.*)(?="\swidth)')

In [70]:
#products = s.find_all('div', attrs={'' : ''}: 'contains(@class, "p13n-sc-truncated")]/text()').getall()

###Rating por estrella X.X de un máximo de 5 estrellas.

In [ ]:
star_ratings_soup = s.find_all('span', attrs={'class' : 'a-icon-alt'})
star_ratings = [star_rating.get_text() for star_rating in star_ratings_soup]
print(f'Nombres guardados: {len(star_ratings)}\n')
star_ratings

###Número de Reviews

In [ ]:
reviews_soup = s.find_all('a', attrs={'class' : 'a-size-small a-link-normal'})
reviews = [review.get_text() for review in reviews_soup]
print(f'Nombres guardados: {len(reviews)}\n')
reviews

###Autores/Compañía

In [ ]:
authors_soup = s.find_all('span', attrs={'class' : 'a-size-small a-color-base'})
authors = [author.get_text() for author in authors_soup]
print(f'Nombres guardados: {len(authors)}\n')
authors

###Edición/Consola
Edición para los libros y para indicar que es de Kindle
Consola para el caso de videojuegos

In [ ]:
editions_soup = s.find_all('span', attrs={'class' : 'a-size-small a-color-secondary'})
editions = [edition.get_text() for edition in editions_soup]
print(f'Nombres guardados: {len(editions)}\n')
editions

#Segmentación de la sopa por cajas. Extracción de boxes.

##Extracción de una sola caja

Resulta que al momento de pedir algunas propiedades, como el rating por estrellas, el número de reviews o el equivalente a la consola, la información se desajusta. Esto se debe a que el "Find_all" no indica si en la caja no se encuentra un dato, solo regresa menos datos. Entonces, suponiendo que falta un dato en el producto 25, nos regresaría un arreglo de 49 datos, en vez de un "NaN", "N/A", "Error", "Not Found" en el puesto 25.

Si pasaramos esta información a un csv, al momento de analizarlo veríamos que la información no coincide del producto 25 (ya que tendría información) hasta el producto 50.

Por ello, este análisis para extraer cada una de las propiedades de forma individual

In [91]:
all_product_info_soup = s.find_all('div', attrs={'class':"a-section a-spacing-none aok-relative"})
print(len(all_product_info_soup))

50


In [ ]:
all_product_info_soup

In [76]:
soup_check = all_product_info_soup[0]
soup_check

<div class="a-section a-spacing-none aok-relative"><div class="a-row a-spacing-none aok-inline-block"><span class="a-size-small aok-float-left zg-badge-body zg-badge-color"><span class="zg-badge-text">#1</span></span><span class="aok-float-left zg-badge-triangle zg-badge-color"></span></div><span class="aok-inline-block zg-item"><a class="a-link-normal" href="/Cero-Inversionista-Domina-juego-dinero-ebook/dp/B08BY3L6C7?_encoding=UTF8&amp;psc=1"><span class="zg-text-center-align"><div class="a-section a-spacing-small"><img alt="De Cero a Inversionista: Domina el juego del dinero" height="200" src="https://images-na.ssl-images-amazon.com/images/I/81w%2BvFNzElL._UX300__PJku-sticker-v7,TopRight,0,-50__BG0,0,0,0_FMpng_AC_UL200_SR200,200_.jpg" width="200"/></div></span>
<div aria-hidden="true" class="p13n-sc-truncate p13n-sc-line-clamp-1" data-rows="1">
            De Cero a Inversionista: Domina el juego del dinero
        </div>
</a><div class="a-row a-size-small"><span class="a-size-small 

Como es posible apreciar en el chunck anterior, se puede interactuar con cada uno de los elementos del arreglo como si fuera una sopa independiente. Por lo que si se hace un ciclo for con cada una de las cajas, sería el mismo efecto.

In [93]:
boxes = all_product_info_soup
cont = 1
for box in boxes:
    if cont == 25:
        print(box)
        cont+=1
    else:
        cont+=1

<div class="a-section a-spacing-none aok-relative"><div class="a-row a-spacing-none aok-inline-block"><span class="a-size-small aok-float-left zg-badge-body zg-badge-color"><span class="zg-badge-text">#25</span></span><span class="aok-float-left zg-badge-triangle zg-badge-color"></span></div><span class="aok-inline-block zg-item"><a class="a-link-normal" href="/verdad-pandemia-Qui%C3%A9n-sido-por-ebook/dp/B089RQCHKQ?_encoding=UTF8&amp;psc=1"><span class="zg-text-center-align"><div class="a-section a-spacing-small"><img alt="La verdad de la pandemia: Quién ha sido y por qué" height="200" src="https://images-na.ssl-images-amazon.com/images/I/81MsVSwOj6L.__BG0,0,0,0_FMpng_AC_UL200_SR200,200_.jpg" width="200"/></div></span>
<div aria-hidden="true" class="p13n-sc-truncate p13n-sc-line-clamp-1" data-rows="1">
            La verdad de la pandemia: Quién ha sido y por qué
        </div>
</a><div class="a-row a-size-small"><span class="a-size-small a-color-base">Cristina Martín Jiménez</span></

##Obtención del ranking

In [88]:
soup_check = all_product_info_soup[0]
ranks_soup = soup_check.find_all('span', attrs={'class':'zg-badge-text'})
rank = ranks_soup[0].get_text()
rank

'#1'

In [95]:
boxes = all_product_info_soup
cont = 1
for box in boxes:
    ##para obtener el número #25
    if cont == 25:
        rank_box = box.find_all('span', attrs={'class':'zg-badge-text'})
        rank = rank_box[0].get_text()
        print(rank)
        cont+=1
    else:
        cont+=1

#25


##Obtención de los links

In [87]:
product_soup = soup_check.find_all('a', attrs={'class' : 'a-link-normal'})
product_s = product_soup[0]
print(product_s)
print('https://www.amazon.com.mx' + product_s.get('href'))


<a class="a-link-normal" href="/Cero-Inversionista-Domina-juego-dinero-ebook/dp/B08BY3L6C7?_encoding=UTF8&amp;psc=1"><span class="zg-text-center-align"><div class="a-section a-spacing-small"><img alt="De Cero a Inversionista: Domina el juego del dinero" height="200" src="https://images-na.ssl-images-amazon.com/images/I/81w%2BvFNzElL._UX300__PJku-sticker-v7,TopRight,0,-50__BG0,0,0,0_FMpng_AC_UL200_SR200,200_.jpg" width="200"/></div></span>
<div aria-hidden="true" class="p13n-sc-truncate p13n-sc-line-clamp-1" data-rows="1">
            De Cero a Inversionista: Domina el juego del dinero
        </div>
</a>
https://www.amazon.com.mx/Cero-Inversionista-Domina-juego-dinero-ebook/dp/B08BY3L6C7?_encoding=UTF8&psc=1
